<a href="https://colab.research.google.com/github/EunSik312/623_chungnam/blob/main/7%EC%9B%9416%EC%9D%BCImageNet%EC%BD%94%EB%93%9C%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

현재 코드에 사용된 함수가 각 어떤 기능을 하는지 공부하기
torch: 딥러닝 핵심 엔진 (텐서, GPU, 자동미분)

torch.nn: 신경망 구축 도구 (레이어, 손실함수, 모델)

torchvision: 컴퓨터 비전 전용 (모델, 변환, 데이터셋)

requests: HTTP 통신 (파일 다운로드, API 호출)

json: 데이터 교환 (설정 파일, 어노테이션)

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import requests
import json
from io import BytesIO
import cv2

#torch.nn:

torch의 구성요소

주요 구성 요소
레이어 (torch.nn):

Linear - 완전연결층 (Fully Connected Layer)

Conv1d, Conv2d, Conv3d - 합성곱층

LSTM, GRU, RNN - 순환신경망층

BatchNorm1d, BatchNorm2d - 배치 정규화

Dropout - 드롭아웃 (정규화 기법)

Embedding - 임베딩층 (범주형 데이터 처리)


활성화 함수:

ReLU, Sigmoid, Tanh, LeakyReLU, GELU

Softmax, LogSoftmax

손실 함수:

CrossEntropyLoss - 교차엔트로피 손실

MSELoss - 평균제곱오차

BCELoss - 이진 교차엔트로피

NLLLoss, L1Loss, SmoothL1Loss

컨테이너:

Sequential - 레이어를 순차적으로 연결

ModuleList - 모듈들의 리스트

ModuleDict - 모듈들의 딕셔너리

#torchvision

torchvision.datasets
미리 준비된 데이터셋들을 쉽게 다운로드하고 사용할 수 있습니다.

torchvision.transforms
이미지 전처리와 데이터 증강을 위한 변환 함수들입니다.

torchvision.models
사전 훈련된 모델들을 제공합니다.

torchvision.utils
시각화와 유틸리티 함수들입니다.

주요 장점:

표준 데이터셋에 쉽게 접근 가능

다양한 이미지 전처리 도구 제공

검증된 사전 훈련 모델들 활용 가능

컴퓨터 비전 작업을 위한 통합된 환경

#requests
Python에서 HTTP 요청을 쉽게 보낼 수 있게 해주는 라이브러리
사용법
1. GET 요청
2. POST 요청
3. 기타 HTTP 메서드

#json
 Python의 내장 모듈로, JSON(JavaScript Object Notation) 데이터 형식을 다루기 위한 기능을 제공

In [ ]:
# =============================================================================
# ImageNet 1000개 클래스 분류기
# =============================================================================

In [ ]:
print("✅ 라이브러리 로드 완료!")
print(f"🔥 PyTorch 버전: {torch.__version__}")
print(f"💻 CUDA 사용 가능: {torch.cuda.is_available()}")

In [ ]:
class ImageNetClassifier:
    """ImageNet 사전 훈련된 모델을 사용한 1000개 클래스 분류기"""


class(클래스) = 붕어빵 툴

object(객체) = 실제 붕어빵


class(클래스)의 구성 요소:

1.attribute(속성): 데이터를 저장하는 variable(변수)

2.method(메서드): 특정 작업을 수행하는 function(함수)


In [ ]:
    def __init__(self, model_name='resnet50'):
        print(f"🧠 {model_name} 모델 로딩 중...")
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"💻 사용 디바이스: {self.device}")

        # 모델 로드
        self.model = self.load_model(model_name)
        self.model.eval()  # 평가 모드

        # ImageNet 클래스 라벨 로드
        self.class_labels = self.load_imagenet_labels()

        # 이미지 전처리 설정(우리가 가지고 있는 사진의 사이즈가 크기 때문에 resize함)
        self.transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                               std=[0.229, 0.224, 0.225])
        ])

        print(f"✅ {model_name} 모델 준비 완료!")
        print(f"📊 분류 가능한 클래스: 1000개")

이 `__init__(self, model_name='resnet50')` method(메서드)는 class(클래스)의 **constructor(생성자)**로, object(객체) 초기화를 담당합니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def __init__(self, model_name='resnet50'):
   ```
   - `model_name`: default parameter(기본 매개변수)로 'resnet50' 설정
   - object(객체) 생성 시 자동 호출

2. **Device selection(디바이스 선택)**:
   ```python
   self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
   ```
   - `torch.cuda.is_available()`: GPU 사용 가능 여부 확인
   - **Conditional expression(조건 표현식)**: GPU 있으면 'cuda', 없으면 'cpu'
   - `self.device`: instance variable(인스턴스 변수)로 device 정보 저장

3. **Model loading(모델 로딩)**:
   ```python
   self.model = self.load_model(model_name)
   self.model.eval()  # 평가 모드
   ```
   - `load_model()`: 앞서 정의한 method(메서드) 호출
   - `eval()`: evaluation mode(평가 모드) 설정 (dropout, batch normalization 비활성화)

4. **Labels loading(레이블 로딩)**:
   ```python
   self.class_labels = self.load_imagenet_labels()
   ```
   - ImageNet 1,000개 class labels(클래스 레이블) 로딩

5. **Image preprocessing pipeline(이미지 전처리 파이프라인)**:
   ```python
   self.transform = transforms.Compose([
       transforms.Resize(256),              # 256x256으로 resize(크기 조정)
       transforms.CenterCrop(224),          # 중앙에서 224x224 crop(자르기)
       transforms.ToTensor(),               # PIL Image → Tensor 변환
       transforms.Normalize(mean=[0.485, 0.456, 0.406],  # RGB 정규화
                          std=[0.229, 0.224, 0.225])
   ])
   ```

6. **Normalization values(정규화 값들)**:
   ```python
   mean=[0.485, 0.456, 0.406],  # ImageNet RGB mean(평균)
   std=[0.229, 0.224, 0.225]    # ImageNet RGB std(표준편차)
   ```
   - ImageNet dataset(데이터셋)에서 계산된 statistical values(통계적 값)
   - pre-trained model(사전 훈련된 모델)과 동일한 normalization(정규화) 적용

**주석 분석:**
- "우리가 가지고 있는 사진의 사이즈가 크기 때문에 resize함" → large image files(큰 이미지 파일) 처리를 위한 resize 필요성

**초기화 순서:**
1. Device 설정 (GPU/CPU)
2. Model 로딩 및 evaluation mode 설정
3. Class labels 로딩
4. Image preprocessing pipeline 설정
5. 완료 메시지 출력

**Instance variables(인스턴스 변수):**
- `self.device`: 사용할 computing device(컴퓨팅 디바이스)
- `self.model`: 로딩된 neural network model(신경망 모델)
- `self.class_labels`: ImageNet class names(클래스 이름들)
- `self.transform`: image preprocessing pipeline(이미지 전처리 파이프라인)

def __init__(self, model_name='resnet50'):

이 함수는 class의 constructor이다

##__init__

Python의 special method(특수 메서드) 또는 magic method(매직 메서드)

object(객체)가 생성될 때 자동으로 호출되는 initialization method(초기화 메서드)

constructor(생성자)라고도 부름tialization method(초기화 메서드)

##self
생성되는 object(객체) 자신을 가리키는 reference(참조)

모든 method(메서드)의 첫 번째 parameter(매개변수)로 사용

instance(인스턴스) 자신을 의미

##model_name='resnet50'

parameter(매개변수) 또는 argument(인수)

In [ ]:
#mobilenet_v2 차선인식할 때 사용함
#다양한 모델 사용
    def load_model(self, model_name):
        """사전 훈련된 모델 로드"""
        models_dict = {
            'resnet50': models.resnet50(pretrained=True),
            'resnet101': models.resnet101(pretrained=True),
            'vgg16': models.vgg16(pretrained=True),
            'vgg19': models.vgg19(pretrained=True),
            'densenet121': models.densenet121(pretrained=True),
            'efficientnet_b0': models.efficientnet_b0(pretrained=True),
            'mobilenet_v2': models.mobilenet_v2(pretrained=True),
            'alexnet': models.alexnet(pretrained=True),
            'inception_v3': models.inception_v3(pretrained=True)
        }

        if model_name not in models_dict:
            print(f"⚠️ {model_name} 모델을 찾을 수 없습니다. ResNet50을 사용합니다.")
            model_name = 'resnet50'

        model = models_dict[model_name]
        return model.to(self.device)


이 `load_model(self, model_name)` method(메서드)는 다양한 pre-trained models(사전 훈련된 모델)을 로딩하는 기능입니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def load_model(self, model_name):
   ```
   - `model_name`: 로드할 model(모델)의 이름을 받는 parameter(매개변수)

2. **Models dictionary(모델 딕셔너리)**:
   ```python
   models_dict = {
       'resnet50': models.resnet50(pretrained=True),
       'resnet101': models.resnet101(pretrained=True),
       # ... 다른 모델들
   }
   ```
   - dictionary(딕셔너리) 형태로 multiple models(여러 모델) 저장
   - `pretrained=True`: ImageNet에서 pre-trained weights(사전 훈련된 가중치) 사용

3. **Available models(사용 가능한 모델들)**:
   - **ResNet**: `resnet50`, `resnet101` - residual networks(잔차 네트워크)
   - **VGG**: `vgg16`, `vgg19` - Visual Geometry Group models(시각 기하학 그룹 모델)
   - **DenseNet**: `densenet121` - densely connected networks(밀집 연결 네트워크)
   - **EfficientNet**: `efficientnet_b0` - efficient neural networks(효율적인 신경망)
   - **MobileNet**: `mobilenet_v2` - mobile-optimized networks(모바일 최적화 네트워크)
   - **AlexNet**: `alexnet` - classic CNN architecture(클래식 CNN 구조)
   - **Inception**: `inception_v3` - Google's Inception architecture(구글의 인셉션 구조)

4. **Error handling(에러 처리)**:
   ```python
   if model_name not in models_dict:
       print(f"⚠️ {model_name} 모델을 찾을 수 없습니다. ResNet50을 사용합니다.")
       model_name = 'resnet50'
   ```
   - invalid model name(잘못된 모델 이름) 시 default(기본값)로 ResNet50 사용

5. **Model loading and device transfer(모델 로딩 및 디바이스 전송)**:
   ```python
   model = models_dict[model_name]
   return model.to(self.device)
   ```
   - 선택된 model(모델) 로딩
   - `to(self.device)`: model(모델)을 specified device(지정된 디바이스)로 이동 (CPU/GPU)

**주석 분석:**
- "mobilenet_v2 차선인식할 때 사용함" → lane detection(차선 인식) project(프로젝트)에서 MobileNet V2 사용
- "다양한 모델 사용" → multiple model architectures(여러 모델 구조) 지원

**사용 예시:**
```python
model = self.load_model('mobilenet_v2')  # MobileNet V2 로딩
model = self.load_model('resnet50')      # ResNet-50 로딩
```

**return value(반환값)**: 지정된 device(디바이스)에 로딩된 PyTorch model(파이토치 모델)

In [ ]:
    def load_imagenet_labels(self):
        """ImageNet 1000개 클래스 라벨 로드"""
        try:
            # ImageNet 클래스 라벨 다운로드, 다양한 차선은 모두 각각 라벨링 해야함
            url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
            response = requests.get(url)
            labels = response.text.strip().split('\n')
            print(f"📋 ImageNet 라벨 로드 완료: {len(labels)}개")
            return labels
        except:
            print("⚠️ 온라인 라벨 로드 실패. 기본 라벨을 사용합니다.")
            # 일부 주요 클래스만 포함한 기본 라벨
            return [f"class_{i}" for i in range(1000)]

이 `load_imagenet_labels(self)` method(메서드)는 ImageNet의 1,000개 class labels(클래스 레이블)을 로딩하는 기능입니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def load_imagenet_labels(self):
   ```
   - parameter(매개변수) 없이 self만 받음
   - return value(반환값)로 labels list(레이블 리스트) 반환

2. **Try-except block(예외 처리 블록)**:
   ```python
   try:
       # 온라인에서 라벨 다운로드 시도
   except:
       # 실패 시 기본 라벨 사용
   ```

3. **Online label loading(온라인 라벨 로딩)**:
   ```python
   url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
   response = requests.get(url)
   labels = response.text.strip().split('\n')
   ```
   - GitHub에서 official ImageNet labels(공식 ImageNet 레이블) 다운로드
   - `strip()`: 앞뒤 whitespace(공백) 제거
   - `split('\n')`: newline character(줄바꿈 문자)로 분할해서 list(리스트)로 변환

4. **Success logging(성공 로깅)**:
   ```python
   print(f"📋 ImageNet 라벨 로드 완료: {len(labels)}개")
   ```
   - f-string formatting(포맷팅)으로 loaded labels count(로드된 레이블 개수) 출력

5. **Fallback mechanism(대체 메커니즘)**:
   ```python
   return [f"class_{i}" for i in range(1000)]
   ```
   - List comprehension(리스트 컴프리헨션)으로 generic labels(일반적인 레이블) 생성
   - network error(네트워크 에러) 시 `class_0`, `class_1`, ..., `class_999` 형태

**주석 분석:**
- "다양한 차선은 모두 각각 라벨링 해야함" → 아마도 traffic/road classification(교통/도로 분류) project(프로젝트)에서 사용

**실제 ImageNet labels 예시:**
- Index 0: 'tench' (텐치, 물고기 종류)
- Index 281: 'tabby cat' (얼룩 고양이)  
- Index 285: 'Egyptian cat' (이집트 고양이)

**return value(반환값)**: 1,000개 string(문자열) 요소를 가진 list(리스트)

In [ ]:
    def load_image(self, image_source):
        """이미지 로드 (파일 경로 또는 URL)"""
        try:
            if isinstance(image_source, str):
                if image_source.startswith('http'):
                    # URL에서 이미지 다운로드
                    response = requests.get(image_source)
                    image = Image.open(BytesIO(response.content)).convert('RGB')
                else:
                    # 로컬 파일
                    image = Image.open(image_source).convert('RGB')
            else:
                # PIL Image 객체
                image = image_source.convert('RGB')

            return image
        except Exception as e:
            print(f"❌ 이미지 로드 실패: {e}")
            return None


이 `load_image(self, image_source)` method(메서드)는 다양한 source(소스)에서 image(이미지)를 로딩하는 기능입니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def load_image(self, image_source):
   ```
   - `image_source`: 이미지 소스 (file path(파일 경로), URL, 또는 PIL Image object(객체))

2. **Input type checking(입력 타입 체크)**:
   ```python
   if isinstance(image_source, str):
   ```
   - `isinstance()`: object(객체)의 type(타입) 확인
   - string(문자열)인지 확인 (file path 또는 URL)

3. **URL handling(URL 처리)**:
   ```python
   if image_source.startswith('http'):
       response = requests.get(image_source)
       image = Image.open(BytesIO(response.content)).convert('RGB')
   ```
   - `startswith('http')`: URL 여부 확인
   - `requests.get()`: HTTP request(요청)로 이미지 다운로드
   - `BytesIO()`: bytes data(바이트 데이터)를 file-like object(파일 유사 객체)로 변환
   - `Image.open()`: PIL Image object(객체) 생성
   - `convert('RGB')`: color mode(색상 모드)를 RGB로 변환

4. **Local file handling(로컬 파일 처리)**:
   ```python
   else:
       image = Image.open(image_source).convert('RGB')
   ```
   - 로컬 file path(파일 경로)에서 직접 이미지 로딩

5. **PIL Image object handling(PIL 이미지 객체 처리)**:
   ```python
   else:
       image = image_source.convert('RGB')
   ```
   - 이미 PIL Image object(객체)인 경우 RGB로만 변환

6. **Error handling(에러 처리)**:
   ```python
   except Exception as e:
       print(f"❌ 이미지 로드 실패: {e}")
       return None
   ```
   - 모든 exception(예외) 처리
   - 실패 시 `None` 반환

**사용 예시:**
```python
# URL에서 로딩
image1 = self.load_image("https://example.com/cat.jpg")

# 로컬 파일에서 로딩
image2 = self.load_image("./images/dog.png")

# PIL Image object에서 로딩
pil_image = Image.open("photo.jpg")
image3 = self.load_image(pil_image)
```

**return value(반환값)**: PIL Image object(객체) 또는 `None` (실패 시)

In [ ]:
   def preprocess_image(self, image):
        """이미지 전처리"""
        if image is None:
            return None

        # PIL Image → Tensor
        tensor = self.transform(image).unsqueeze(0)  # 배치 차원 추가
        return tensor.to(self.device)


이 `preprocess_image(self, image)` method(메서드)는 raw image(원본 이미지)를 model input(모델 입력) 형태로 변환하는 기능입니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def preprocess_image(self, image):
   ```
   - `image`: PIL Image object(객체) 또는 None을 받는 parameter(매개변수)

2. **Null check(널 체크)**:
   ```python
   if image is None:
       return None
   ```
   - input validation(입력 검증)으로 None 값 처리

3. **Image transformation(이미지 변환)**:
   ```python
   tensor = self.transform(image).unsqueeze(0)  # 배치 차원 추가
   ```
   - `self.transform(image)`: constructor(생성자)에서 정의한 preprocessing pipeline(전처리 파이프라인) 적용
   - `unsqueeze(0)`: batch dimension(배치 차원) 추가
   - shape 변화: `[C, H, W]` → `[1, C, H, W]`

4. **Device transfer(디바이스 전송)**:
   ```python
   return tensor.to(self.device)
   ```
   - tensor(텐서)를 지정된 device(디바이스, CPU/GPU)로 이동
   - model(모델)과 동일한 device에 위치시킴

**Transformation pipeline(변환 파이프라인) 적용:**
- `Resize(256)`: 이미지 크기를 256x256으로 조정
- `CenterCrop(224)`: 중앙에서 224x224 크기로 crop(자르기)
- `ToTensor()`: PIL Image → PyTorch Tensor 변환, [0,255] → [0,1] 정규화
- `Normalize()`: ImageNet statistics(통계)로 정규화

**Tensor shape(텐서 형태):**
- Input: PIL Image (H, W, C)
- After transform: Tensor (C, H, W) = (3, 224, 224)
- After unsqueeze: Tensor (1, C, H, W) = (1, 3, 224, 224)

**return value(반환값)**:
- Success: preprocessed tensor(전처리된 텐서) on specified device(지정된 디바이스)
- Failure: None

**역할:**
- Raw image(원본 이미지)를 neural network(신경망) input format(입력 형식)으로 변환
- Model inference(모델 추론)을 위한 준비 작업

In [ ]:
    def predict(self, image_source, top_k=5):
        """이미지 분류 예측"""
        print(f"🔍 이미지 분석 중...")

        # 이미지 로드
        image = self.load_image(image_source)
        if image is None:
            return None

        original_image = image.copy()

        # 전처리
        input_tensor = self.preprocess_image(image)
        if input_tensor is None:
            return None

        # 예측 torch.no_grad(): 이런 모르는 용어 정리하기, 함수별로 쪼개서 설명, 주석 달기
        with torch.no_grad():
            outputs = self.model(input_tensor)
            probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

        # Top-K 결과 추출
        top_prob, top_indices = torch.topk(probabilities, top_k)

        results = []
        for i in range(top_k):
            class_idx = top_indices[i].item()
            prob = top_prob[i].item()
            class_name = self.class_labels[class_idx]
            results.append({
                'rank': i + 1,
                'class_index': class_idx,
                'class_name': class_name,
                'probability': prob,
                'percentage': prob * 100
            })

        return {
            'original_image': original_image,
            'predictions': results,
            'model_info': {
                'model_name': self.model.__class__.__name__,
                'device': str(self.device),
                'total_classes': len(self.class_labels)
            }
        }


이 `predict(self, image_source, top_k=5)` method(메서드)는 이미지 분류 예측을 수행하는 main function(메인 함수)입니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def predict(self, image_source, top_k=5):
   ```
   - `image_source`: 이미지 소스 (file path, URL, PIL Image)
   - `top_k=5`: 상위 몇 개 결과를 반환할지 결정하는 parameter(매개변수)

2. **Image loading(이미지 로딩)**:
   ```python
   image = self.load_image(image_source)
   if image is None:
       return None
   original_image = image.copy()
   ```
   - `load_image()`: 이미지 로딩 method(메서드) 호출
   - `copy()`: original image(원본 이미지) 백업 저장

3. **Image preprocessing(이미지 전처리)**:
   ```python
   input_tensor = self.preprocess_image(image)
   if input_tensor is None:
       return None
   ```
   - `preprocess_image()`: 이미지를 model input format(모델 입력 형식)으로 변환

4. **Model inference(모델 추론)**:
   ```python
   with torch.no_grad():
       outputs = self.model(input_tensor)
       probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
   ```
   - `torch.no_grad()`: gradient calculation(기울기 계산) 비활성화, memory efficiency(메모리 효율성) 향상
   - `self.model(input_tensor)`: forward pass(순전파) 실행
   - `softmax()`: raw output(원시 출력)을 probability distribution(확률 분포)로 변환
   - `dim=0`: 첫 번째 차원(class dimension)에서 softmax 적용

5. **Top-K results extraction(상위 K개 결과 추출)**:
   ```python
   top_prob, top_indices = torch.topk(probabilities, top_k)
   ```
   - `torch.topk()`: 가장 높은 확률의 top_k개 값과 index(인덱스) 반환
   - `top_prob`: 상위 확률값들
   - `top_indices`: 해당 class index(클래스 인덱스)들

6. **Results formatting(결과 포맷팅)**:
   ```python
   results = []
   for i in range(top_k):
       class_idx = top_indices[i].item()
       prob = top_prob[i].item()
       class_name = self.class_labels[class_idx]
       results.append({
           'rank': i + 1,
           'class_index': class_idx,
           'class_name': class_name,
           'probability': prob,
           'percentage': prob * 100
       })
   ```
   - `item()`: tensor scalar(텐서 스칼라)를 Python number(파이썬 숫자)로 변환
   - dictionary format(딕셔너리 형식)으로 결과 구성

7. **Return value(반환값)**:
   ```python
   return {
       'original_image': original_image,
       'predictions': results,
       'model_info': {
           'model_name': self.model.__class__.__name__,
           'device': str(self.device),
           'total_classes': len(self.class_labels)
       }
   }
   ```
   - structured output(구조화된 출력): 원본 이미지, 예측 결과, 모델 정보 포함

**주요 함수 설명:**
- `torch.no_grad()`: inference(추론) 시 gradient computation(기울기 계산) 비활성화
- `torch.nn.functional.softmax()`: logits을 probability(확률)로 변환
- `torch.topk()`: tensor에서 가장 큰 k개 값과 index 추출
- `item()`: single-element tensor를 Python scalar로 변환

In [ ]:
def visualize_results(self, results, show_top_k=5):
        """결과 시각화"""
        if results is None:
            print("❌ 결과가 없습니다.")
            return

        # 결과 출력
        print(f"\n🎯 ImageNet 분류 결과 (Top {show_top_k}):")
        print("=" * 60)

        predictions = results['predictions'][:show_top_k]

        for pred in predictions:
            print(f"{pred['rank']}. {pred['class_name']:<30} {pred['percentage']:.2f}%")

        print("=" * 60)

        # 시각화
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # 원본 이미지
        ax1.imshow(results['original_image'])
        ax1.set_title('Original Image', fontsize=14, fontweight='bold')
        ax1.axis('off')

        # 예측 결과 막대 그래프
        class_names = [pred['class_name'][:20] for pred in predictions]  # 이름 길이 제한
        probabilities = [pred['percentage'] for pred in predictions]
        colors = plt.cm.viridis(np.linspace(0, 1, len(predictions)))

        bars = ax2.barh(range(len(predictions)), probabilities, color=colors)
        ax2.set_yticks(range(len(predictions)))
        ax2.set_yticklabels(class_names)
        ax2.set_xlabel('Confidence (%)')
        ax2.set_title(f'Top {show_top_k} Predictions', fontsize=14, fontweight='bold')
        ax2.invert_yaxis()  # 높은 확률이 위로

        # 막대에 퍼센트 표시
        for i, (bar, prob) in enumerate(zip(bars, probabilities)):
            ax2.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
                    f'{prob:.1f}%', va='center', fontweight='bold')

        # 전체 제목
        best_prediction = predictions[0]
        plt.suptitle(f'🏆 Best Prediction: {best_prediction["class_name"]} ({best_prediction["percentage"]:.1f}%)',
                    fontsize=16, fontweight='bold')

        plt.tight_layout()
        plt.show()

        # 모델 정보
        model_info = results['model_info']
        print(f"\n📊 모델 정보:")
        print(f"   🧠 모델: {model_info['model_name']}")
        print(f"   💻 디바이스: {model_info['device']}")
        print(f"   📋 총 클래스 수: {model_info['total_classes']}")


이 `visualize_results(self, results, show_top_k=5)` method(메서드)는 prediction results(예측 결과)를 시각화하는 기능입니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def visualize_results(self, results, show_top_k=5):
   ```
   - `results`: predict() method(메서드)에서 반환된 dictionary(딕셔너리)
   - `show_top_k=5`: 표시할 상위 결과 개수

2. **Input validation(입력 검증)**:
   ```python
   if results is None:
       print("❌ 결과가 없습니다.")
       return
   ```
   - None 값 처리

3. **Text output(텍스트 출력)**:
   ```python
   predictions = results['predictions'][:show_top_k]
   for pred in predictions:
       print(f"{pred['rank']}. {pred['class_name']:<30} {pred['percentage']:.2f}%")
   ```
   - `[:show_top_k]`: list slicing(리스트 슬라이싱)으로 상위 k개만 선택
   - `:<30`: left-aligned(왼쪽 정렬) 30자 width(너비)
   - `:.2f`: 소수점 2자리 formatting(포맷팅)

4. **Subplot creation(서브플롯 생성)**:
   ```python
   fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
   ```
   - `subplots(1, 2)`: 1행 2열 layout(레이아웃)
   - `figsize=(15, 6)`: figure size(그림 크기) 설정

5. **Original image display(원본 이미지 표시)**:
   ```python
   ax1.imshow(results['original_image'])
   ax1.set_title('Original Image', fontsize=14, fontweight='bold')
   ax1.axis('off')
   ```
   - `imshow()`: image display(이미지 표시)
   - `axis('off')`: axis labels(축 라벨) 제거

6. **Bar chart creation(막대 그래프 생성)**:
   ```python
   class_names = [pred['class_name'][:20] for pred in predictions]
   probabilities = [pred['percentage'] for pred in predictions]
   colors = plt.cm.viridis(np.linspace(0, 1, len(predictions)))
   ```
   - `[:20]`: string slicing(문자열 슬라이싱)으로 이름 길이 제한
   - `plt.cm.viridis()`: colormap(컬러맵) 사용
   - `np.linspace()`: 균등 간격 색상 생성

7. **Horizontal bar chart(수평 막대 그래프)**:
   ```python
   bars = ax2.barh(range(len(predictions)), probabilities, color=colors)
   ax2.set_yticks(range(len(predictions)))
   ax2.set_yticklabels(class_names)
   ax2.invert_yaxis()  # 높은 확률이 위로
   ```
   - `barh()`: horizontal bar chart(수평 막대 그래프)
   - `invert_yaxis()`: y축 순서 뒤집기

8. **Text annotations(텍스트 주석)**:
   ```python
   for i, (bar, prob) in enumerate(zip(bars, probabilities)):
       ax2.text(bar.get_width() + 0.5, bar.get_y() + bar.get_height()/2,
               f'{prob:.1f}%', va='center', fontweight='bold')
   ```
   - `enumerate()`: index와 value 동시 반복
   - `zip()`: 두 list를 pair(쌍)로 결합
   - `va='center'`: vertical alignment(수직 정렬)

9. **Title and layout(제목 및 레이아웃)**:
   ```python
   best_prediction = predictions[0]
   plt.suptitle(f'🏆 Best Prediction: {best_prediction["class_name"]} ({best_prediction["percentage"]:.1f}%)',
               fontsize=16, fontweight='bold')
   plt.tight_layout()
   plt.show()
   ```
   - `suptitle()`: main title(메인 제목) 설정
   - `tight_layout()`: automatic spacing(자동 간격) 조정
   - `show()`: plot display(플롯 표시)

10. **Model information display(모델 정보 표시)**:
    ```python
    model_info = results['model_info']
    print(f"   🧠 모델: {model_info['model_name']}")
    print(f"   💻 디바이스: {model_info['device']}")
    print(f"   📋 총 클래스 수: {model_info['total_classes']}")
    ```
    - metadata(메타데이터) 출력

**시각화 구성:**
- Left panel(왼쪽 패널): original image(원본 이미지)
- Right panel(오른쪽 패널): confidence scores(신뢰도 점수) 막대 그래프
- Text output(텍스트 출력): ranking과 percentage(퍼센트) 정보

In [ ]:
    def compare_models(self, image_source, model_names=['resnet50', 'vgg16', 'efficientnet_b0']):
        """여러 모델 성능 비교"""
        print(f"🔄 다중 모델 비교 시작...")

        original_model = self.model
        original_name = original_model.__class__.__name__

        results_comparison = []

        for model_name in model_names:
            print(f"\n🧠 {model_name} 모델로 예측 중...")

            # 모델 변경
            self.model = self.load_model(model_name)
            self.model.eval()

            # 예측
            result = self.predict(image_source, top_k=3)
            if result:
                best_pred = result['predictions'][0]
                results_comparison.append({
                    'model_name': model_name,
                    'best_class': best_pred['class_name'],
                    'confidence': best_pred['percentage'],
                    'top3': result['predictions']
                })

        # 원래 모델 복원
        self.model = original_model

        # 비교 결과 시각화
        if results_comparison:
            self.visualize_model_comparison(results_comparison, image_source)

        return results_comparison


이 `compare_models(self, image_source, model_names=['resnet50', 'vgg16', 'efficientnet_b0'])` method(메서드)는 multiple models(여러 모델)의 성능을 비교하는 기능입니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def compare_models(self, image_source, model_names=['resnet50', 'vgg16', 'efficientnet_b0']):
   ```
   - `image_source`: 비교할 이미지 소스
   - `model_names`: 비교할 model list(모델 리스트), default(기본값) 3개 모델

2. **Current model backup(현재 모델 백업)**:
   ```python
   original_model = self.model
   original_name = original_model.__class__.__name__
   ```
   - 현재 사용 중인 model(모델) 백업 저장
   - `__class__.__name__`: class name(클래스 이름) 추출

3. **Results container(결과 컨테이너)**:
   ```python
   results_comparison = []
   ```
   - 각 model(모델)의 결과를 저장할 empty list(빈 리스트)

4. **Model iteration(모델 반복)**:
   ```python
   for model_name in model_names:
       print(f"\n🧠 {model_name} 모델로 예측 중...")
   ```
   - 각 model(모델)에 대해 순차적으로 처리

5. **Model switching(모델 교체)**:
   ```python
   self.model = self.load_model(model_name)
   self.model.eval()
   ```
   - `load_model()`: 새로운 model(모델) 로딩
   - `eval()`: evaluation mode(평가 모드) 설정

6. **Prediction execution(예측 실행)**:
   ```python
   result = self.predict(image_source, top_k=3)
   if result:
       best_pred = result['predictions'][0]
   ```
   - `predict()`: 각 model(모델)로 예측 수행
   - `top_k=3`: 상위 3개 결과만 추출
   - `best_pred`: 최고 confidence(신뢰도) 결과

7. **Results aggregation(결과 집계)**:
   ```python
   results_comparison.append({
       'model_name': model_name,
       'best_class': best_pred['class_name'],
       'confidence': best_pred['percentage'],
       'top3': result['predictions']
   })
   ```
   - 각 model(모델)의 결과를 dictionary(딕셔너리) 형태로 저장
   - `best_class`: 최고 예측 class(클래스)
   - `confidence`: 신뢰도 percentage(퍼센트)
   - `top3`: 상위 3개 전체 결과

8. **Model restoration(모델 복원)**:
   ```python
   self.model = original_model
   ```
   - 원래 사용하던 model(모델)로 복원
   - state preservation(상태 보존)

9. **Visualization call(시각화 호출)**:
   ```python
   if results_comparison:
       self.visualize_model_comparison(results_comparison, image_source)
   ```
   - 결과가 있으면 visualization method(시각화 메서드) 호출

10. **Return value(반환값)**:
    ```python
    return results_comparison
    ```
    - 모든 model(모델)의 비교 결과 반환

**Method workflow(메서드 워크플로우):**
1. 현재 model(모델) 백업
2. 각 model(모델)에 대해 순차적으로:
   - Model loading(모델 로딩)
   - Prediction execution(예측 실행)
   - Results storage(결과 저장)
3. 원래 model(모델) 복원
4. 결과 시각화
5. 비교 결과 반환

**Data structure(데이터 구조):**
- `results_comparison`: list of dictionaries(딕셔너리 리스트)
- 각 dictionary(딕셔너리): model name, best prediction, confidence, top-3 results 포함

In [ ]:
    def visualize_model_comparison(self, results_comparison, image_source):
        """모델 비교 결과 시각화"""
        # 이미지 로드
        image = self.load_image(image_source)

        fig, axes = plt.subplots(2, 2, figsize=(16, 12))

        # 원본 이미지
        axes[0, 0].imshow(image)
        axes[0, 0].set_title('Original Image', fontsize=14, fontweight='bold')
        axes[0, 0].axis('off')

        # 각 모델별 Top 1 결과
        model_names = [result['model_name'] for result in results_comparison]
        confidences = [result['confidence'] for result in results_comparison]
        predictions = [result['best_class'][:15] for result in results_comparison]  # 이름 단축

        colors = ['red', 'blue', 'green', 'orange', 'purple'][:len(model_names)]

        bars = axes[0, 1].bar(model_names, confidences, color=colors, alpha=0.7)
        axes[0, 1].set_title('Model Confidence Comparison', fontsize=14, fontweight='bold')
        axes[0, 1].set_ylabel('Confidence (%)')
        axes[0, 1].tick_params(axis='x', rotation=45)

        # 막대에 예측 클래스와 확률 표시
        for bar, pred, conf in zip(bars, predictions, confidences):
            axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                           f'{pred}\n{conf:.1f}%', ha='center', va='bottom', fontsize=10)

        # 상세 비교 테이블
        axes[1, 0].axis('off')
        table_data = []
        for result in results_comparison:
            row = [result['model_name'], result['best_class'][:20], f"{result['confidence']:.1f}%"]
            table_data.append(row)

        table = axes[1, 0].table(cellText=table_data,
                                colLabels=['Model', 'Best Prediction', 'Confidence'],
                                cellLoc='center',
                                loc='center')
        table.auto_set_font_size(False)
        table.set_fontsize(10)
        table.scale(1.2, 1.5)
        axes[1, 0].set_title('Detailed Comparison', fontsize=14, fontweight='bold')

        # 모델별 Top 3 비교
        axes[1, 1].axis('off')
        y_pos = 0.9
        for result in results_comparison:
            axes[1, 1].text(0.05, y_pos, f"🧠 {result['model_name']}:", fontweight='bold', fontsize=12)
            y_pos -= 0.08
            for i, pred in enumerate(result['top3']):
                axes[1, 1].text(0.1, y_pos, f"{i+1}. {pred['class_name'][:25]} ({pred['percentage']:.1f}%)",
                               fontsize=10)
                y_pos -= 0.06
            y_pos -= 0.02

        axes[1, 1].set_xlim(0, 1)
        axes[1, 1].set_ylim(0, 1)
        axes[1, 1].set_title('Top 3 Predictions per Model', fontsize=14, fontweight='bold')

        plt.tight_layout()
        plt.show()


네, 이해했습니다. 앞으로 이런 형식으로 답변드리겠습니다.

이 `compare_models()` (모델 비교) method(메서드)는 multiple models(여러 모델)의 성능을 비교하는 기능입니다.

**코드 분석:**

1. **Method signature(메서드 서명)**:
   ```python
   def compare_models(self, image_source, model_names=['resnet50', 'vgg16', 'efficientnet_b0']):
   ```
   - `image_source` (이미지 소스): 비교할 이미지 소스
   - `model_names` (모델 이름들): 비교할 model list(모델 리스트), default(기본값) 3개 모델

2. **Current model backup(현재 모델 백업)**:
   ```python
   original_model = self.model
   original_name = original_model.__class__.__name__
   ```
   - 현재 사용 중인 model(모델) 백업 저장
   - `__class__.__name__`: class name(클래스 이름) 추출

3. **Results container(결과 컨테이너)**:
   ```python
   results_comparison = []
   ```
   - 각 model(모델)의 결과를 저장할 empty list(빈 리스트)

4. **Model iteration(모델 반복)**:
   ```python
   for model_name in model_names:
       print(f"\n🧠 {model_name} 모델로 예측 중...")
   ```
   - 각 model(모델)에 대해 순차적으로 처리

5. **Model switching(모델 교체)**:
   ```python
   self.model = self.load_model(model_name)
   self.model.eval()
   ```
   - `load_model()` (모델 로드): 새로운 model(모델) 로딩
   - `eval()` (평가 모드): evaluation mode(평가 모드) 설정

6. **Prediction execution(예측 실행)**:
   ```python
   result = self.predict(image_source, top_k=3)
   if result:
       best_pred = result['predictions'][0]
   ```
   - `predict()` (예측): 각 model(모델)로 예측 수행
   - `top_k=3`: 상위 3개 결과만 추출
   - `best_pred` (최고 예측): 최고 confidence(신뢰도) 결과

7. **Results aggregation(결과 집계)**:
   ```python
   results_comparison.append({
       'model_name': model_name,
       'best_class': best_pred['class_name'],
       'confidence': best_pred['percentage'],
       'top3': result['predictions']
   })
   ```
   - 각 model(모델)의 결과를 dictionary(딕셔너리) 형태로 저장
   - `best_class` (최고 클래스): 최고 예측 class(클래스)
   - `confidence` (신뢰도): 신뢰도 percentage(퍼센트)
   - `top3` (상위 3개): 상위 3개 전체 결과

8. **Model restoration(모델 복원)**:
   ```python
   self.model = original_model
   ```
   - 원래 사용하던 model(모델)로 복원
   - state preservation(상태 보존)

9. **Visualization call(시각화 호출)**:
   ```python
   if results_comparison:
       self.visualize_model_comparison(results_comparison, image_source)
   ```
   - 결과가 있으면 visualization method(시각화 메서드) 호출

10. **Return value(반환값)**:
    ```python
    return results_comparison
    ```
    - 모든 model(모델)의 비교 결과 반환

**Method workflow(메서드 워크플로우):**
1. 현재 model(모델) 백업
2. 각 model(모델)에 대해 순차적으로:
   - Model loading(모델 로딩)
   - Prediction execution(예측 실행)
   - Results storage(결과 저장)
3. 원래 model(모델) 복원
4. 결과 시각화
5. 비교 결과 반환

**Data structure(데이터 구조):**
- `results_comparison`: list of dictionaries(딕셔너리 리스트)
- 각 dictionary(딕셔너리): model name, best prediction, confidence, top-3 results 포함

In [ ]:
# =============================================================================
# 실행 함수들
# =============================================================================

In [ ]:
def classify_uploaded_image():
    """파일 업로드 후 ImageNet 분류"""
    print("🚀 ImageNet 1000개 클래스 분류기 시작!")
    print("=" * 60)

    # 파일 업로드
    print("📁 이미지를 업로드해주세요...")
    print("🎯 1000개 클래스 중에서 가장 유사한 객체를 찾아드립니다!")

    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        print("❌ 파일이 업로드되지 않았습니다!")
        return None

    filename = list(uploaded.keys())[0]
    print(f"📷 업로드된 파일: {filename}")

    try:
        # 분류기 생성
        classifier = ImageNetClassifier(model_name='resnet50')

        # 예측
        results = classifier.predict(filename, top_k=10)

        # 결과 시각화
        classifier.visualize_results(results, show_top_k=5)

        print("\n✅ ImageNet 분류 완료!")
        return results

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None


이 `classify_uploaded_image()` (업로드된 이미지 분류) function(함수)는 **Google Colab에서 파일을 업로드하여 ImageNet 분류를 수행**하는 기능입니다.

**코드 분석:**

1. **Function signature(함수 서명)**:
   ```python
   def classify_uploaded_image():
   ```
   - parameter(매개변수) 없이 독립적으로 실행되는 standalone function(독립 함수)

2. **Interface initialization(인터페이스 초기화)**:
   ```python
   print("🚀 ImageNet 1000개 클래스 분류기 시작!")
   print("=" * 60)
   ```
   - user interface(사용자 인터페이스) 시작 메시지 출력
   - visual separator(시각적 구분선) 생성

3. **File upload prompt(파일 업로드 안내)**:
   ```python
   print("📁 이미지를 업로드해주세요...")
   print("🎯 1000개 클래스 중에서 가장 유사한 객체를 찾아드립니다!")
   ```
   - user instruction(사용자 지시사항) 출력
   - ImageNet 1000 classes(클래스) 설명

4. **Google Colab file upload(구글 콜랩 파일 업로드)**:
   ```python
   from google.colab import files
   uploaded = files.upload()
   ```
   - `files.upload()` (파일 업로드): Colab의 file upload widget(파일 업로드 위젯) 호출
   - `uploaded` (업로드된 파일): dictionary format(딕셔너리 형태)의 uploaded files(업로드된 파일들)

5. **Upload validation(업로드 검증)**:
   ```python
   if not uploaded:
       print("❌ 파일이 업로드되지 않았습니다!")
       return None
   ```
   - empty upload(빈 업로드) 확인
   - error handling(오류 처리)으로 early return(조기 반환)

6. **Filename extraction(파일명 추출)**:
   ```python
   filename = list(uploaded.keys())[0]
   print(f"📷 업로드된 파일: {filename}")
   ```
   - `uploaded.keys()`: uploaded dictionary(업로드된 딕셔너리)의 keys(키들) 추출
   - 첫 번째 file(파일)의 name(이름) 선택

7. **Classification workflow(분류 워크플로우)**:
   ```python
   try:
       # 분류기 생성
       classifier = ImageNetClassifier(model_name='resnet50')
       
       # 예측
       results = classifier.predict(filename, top_k=10)
       
       # 결과 시각화
       classifier.visualize_results(results, show_top_k=5)
   ```
   - `ImageNetClassifier()` (ImageNet 분류기): ResNet-50 model(모델)로 classifier instance(분류기 인스턴스) 생성
   - `predict()` (예측): `top_k=10`으로 상위 10개 결과 예측
   - `visualize_results()` (결과 시각화): 상위 5개 결과만 시각화

8. **Success message(성공 메시지)**:
   ```python
   print("\n✅ ImageNet 분류 완료!")
   return results
   ```
   - completion message(완료 메시지) 출력
   - prediction results(예측 결과) 반환

9. **Exception handling(예외 처리)**:
   ```python
   except Exception as e:
       print(f"❌ 오류 발생: {e}")
       return None
   ```
   - generic exception(일반 예외) 처리
   - error message(오류 메시지) 출력 후 `None` 반환

**Function workflow(함수 워크플로우):**
1. User interface(사용자 인터페이스) 초기화
2. File upload(파일 업로드) 실행
3. Upload validation(업로드 검증)
4. Filename extraction(파일명 추출)
5. Classifier initialization(분류기 초기화)
6. Image prediction(이미지 예측)
7. Results visualization(결과 시각화)
8. Success/error handling(성공/오류 처리)

**Key features(주요 특징):**
- **Google Colab integration(구글 콜랩 통합)**: Colab environment(콜랩 환경)에 최적화
- **Interactive workflow(대화형 워크플로우)**: 단계별 사용자 안내
- **Error handling(오류 처리)**: robust exception handling(견고한 예외 처리)
- **Visual feedback(시각적 피드백)**: 이모지와 formatting(서식)을 통한 사용자 경험 향상

**Return value(반환값):**
- Success case(성공 시): prediction results dictionary(예측 결과 딕셔너리)
- Failure case(실패 시): `None`

In [ ]:
def compare_multiple_models():
    """여러 모델로 동일 이미지 비교"""
    print("🔄 다중 모델 성능 비교!")
    print("=" * 60)

    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        print("❌ 파일이 업로드되지 않았습니다!")
        return None

    filename = list(uploaded.keys())[0]
    print(f"📷 업로드된 파일: {filename}")

    try:
        # 분류기 생성
        classifier = ImageNetClassifier()

        # 여러 모델 비교
        models_to_compare = ['resnet50', 'vgg16', 'efficientnet_b0', 'mobilenet_v2']
        comparison_results = classifier.compare_models(filename, models_to_compare)

        print("\n✅ 다중 모델 비교 완료!")
        return comparison_results

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None


이 `compare_multiple_models()` (여러 모델 비교) function(함수)는 **동일한 이미지에 대해 여러 딥러닝 모델들의 성능을 비교**하는 기능입니다.

**코드 분석:**

1. **Function signature(함수 서명)**:
   ```python
   def compare_multiple_models():
   ```
   - parameter(매개변수) 없는 standalone function(독립 함수)
   - multi-model comparison(다중 모델 비교) 전용

2. **Interface initialization(인터페이스 초기화)**:
   ```python
   print("🔄 다중 모델 성능 비교!")
   print("=" * 60)
   ```
   - comparison workflow(비교 워크플로우) 시작 메시지
   - visual separator(시각적 구분선) 생성

3. **File upload process(파일 업로드 과정)**:
   ```python
   from google.colab import files
   uploaded = files.upload()
   ```
   - `files.upload()` (파일 업로드): Colab file upload widget(콜랩 파일 업로드 위젯) 호출
   - `uploaded` (업로드된 파일): uploaded files dictionary(업로드된 파일 딕셔너리)

4. **Upload validation(업로드 검증)**:
   ```python
   if not uploaded:
       print("❌ 파일이 업로드되지 않았습니다!")
       return None
   ```
   - empty upload check(빈 업로드 확인)
   - early termination(조기 종료) with error message(오류 메시지)

5. **Filename extraction(파일명 추출)**:
   ```python
   filename = list(uploaded.keys())[0]
   print(f"📷 업로드된 파일: {filename}")
   ```
   - `uploaded.keys()[0]`: 첫 번째 uploaded file(업로드된 파일)의 name(이름) 추출
   - confirmation message(확인 메시지) 출력

6. **Classifier initialization(분류기 초기화)**:
   ```python
   classifier = ImageNetClassifier()
   ```
   - `ImageNetClassifier()` (ImageNet 분류기): default model(기본 모델)로 classifier instance(분류기 인스턴스) 생성
   - 내부에서 model switching(모델 교체) 수행

7. **Model comparison setup(모델 비교 설정)**:
   ```python
   models_to_compare = ['resnet50', 'vgg16', 'efficientnet_b0', 'mobilenet_v2']
   ```
   - `models_to_compare` (비교할 모델들): 4개의 different architectures(다른 아키텍처) 정의
   - ResNet-50, VGG-16, EfficientNet-B0, MobileNet-V2 포함

8. **Model comparison execution(모델 비교 실행)**:
   ```python
   comparison_results = classifier.compare_models(filename, models_to_compare)
   ```
   - `compare_models()` (모델 비교): 각 model(모델)에 대해 순차적으로 prediction(예측) 수행
   - `comparison_results` (비교 결과): 모든 model results(모델 결과)를 담은 list(리스트)

9. **Success completion(성공 완료)**:
   ```python
   print("\n✅ 다중 모델 비교 완료!")
   return comparison_results
   ```
   - completion message(완료 메시지) 출력
   - comprehensive comparison results(종합 비교 결과) 반환

10. **Exception handling(예외 처리)**:
    ```python
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None
    ```
    - generic error handling(일반 오류 처리)
    - error message(오류 메시지) 출력 후 `None` 반환

**Function workflow(함수 워크플로우):**
1. User interface(사용자 인터페이스) 초기화
2. File upload(파일 업로드) 실행
3. Upload validation(업로드 검증)
4. Filename extraction(파일명 추출)
5. Classifier initialization(분류기 초기화)
6. Model list(모델 리스트) 정의
7. Multi-model comparison(다중 모델 비교) 실행
8. Results compilation(결과 집계)
9. Success/error handling(성공/오류 처리)

**Key differences from single model(단일 모델과의 주요 차이점):**
- **Multiple architectures(다중 아키텍처)**: 4개의 서로 다른 model types(모델 유형)
- **Performance comparison(성능 비교)**: confidence scores(신뢰도 점수) 비교
- **Comprehensive visualization(종합 시각화)**: 모든 모델의 결과를 한 화면에 표시

**Model architectures compared(비교되는 모델 아키텍처):**
- `resnet50`: Residual Network(잔차 네트워크) - 50 layers(층)
- `vgg16`: Visual Geometry Group(비주얼 지오메트리 그룹) - 16 layers(층)
- `efficientnet_b0`: Efficient Neural Network(효율적 신경망) - Base model(기본 모델)
- `mobilenet_v2`: Mobile Network(모바일 네트워크) - Version 2(버전 2)

**Return value(반환값):**
- Success case(성공 시): list of dictionaries(딕셔너리 리스트) containing model comparison results(모델 비교 결과)
- Failure case(실패 시): `None`

In [ ]:
def batch_classify():
    """여러 이미지 일괄 업로드 분류"""
    print("📁 여러 이미지 일괄 분류!")
    print("=" * 60)

    from google.colab import files
    uploaded = files.upload()

    if not uploaded:
        print("❌ 파일이 업로드되지 않았습니다!")
        return None

    try:
        classifier = ImageNetClassifier()
        results_list = []

        for filename in uploaded.keys():
            print(f"\n🔍 {filename} 분석 중...")
            results = classifier.predict(filename, top_k=5)

            if results:
                # 간단한 결과 출력
                best_pred = results['predictions'][0]
                print(f"🏆 {filename}: {best_pred['class_name']} ({best_pred['percentage']:.1f}%)")
                results_list.append((filename, best_pred['class_name'], best_pred['percentage']))

        # 전체 결과 요약
        print(f"\n{'='*60}")
        print("📋 일괄 분류 결과 요약")
        print(f"{'='*60}")
        for filename, pred_class, confidence in results_list:
            print(f"📷 {filename:25} → {pred_class:20} ({confidence:.1f}%)")

        return results_list

    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None


이 `batch_classify()` (일괄 분류) function(함수)는 **여러 이미지를 한 번에 업로드하여 일괄적으로 분류**하는 기능입니다.

**코드 분석:**

1. **Function signature(함수 서명)**:
   ```python
   def batch_classify():
   ```
   - parameter(매개변수) 없는 standalone function(독립 함수)
   - batch processing(일괄 처리) 전용

2. **Interface initialization(인터페이스 초기화)**:
   ```python
   print("📁 여러 이미지 일괄 분류!")
   print("=" * 60)
   ```
   - batch classification(일괄 분류) 시작 메시지
   - visual separator(시각적 구분선) 생성

3. **Multiple file upload(다중 파일 업로드)**:
   ```python
   from google.colab import files
   uploaded = files.upload()
   ```
   - `files.upload()` (파일 업로드): Colab에서 multiple files(여러 파일) 업로드 지원
   - `uploaded` (업로드된 파일들): filename-content dictionary(파일명-내용 딕셔너리)

4. **Upload validation(업로드 검증)**:
   ```python
   if not uploaded:
       print("❌ 파일이 업로드되지 않았습니다!")
       return None
   ```
   - empty upload check(빈 업로드 확인)
   - early termination(조기 종료) with error handling(오류 처리)

5. **Batch processing setup(일괄 처리 설정)**:
   ```python
   classifier = ImageNetClassifier()
   results_list = []
   ```
   - `ImageNetClassifier()` (ImageNet 분류기): single classifier instance(단일 분류기 인스턴스) 생성
   - `results_list` (결과 리스트): 모든 image results(이미지 결과)를 저장할 container(컨테이너)

6. **File iteration loop(파일 반복 루프)**:
   ```python
   for filename in uploaded.keys():
       print(f"\n🔍 {filename} 분석 중...")
       results = classifier.predict(filename, top_k=5)
   ```
   - `uploaded.keys()`: 업로드된 모든 filenames(파일명) 순회
   - `predict()` (예측): 각 image(이미지)에 대해 `top_k=5`로 상위 5개 결과 예측
   - progress indicator(진행 상황 표시) 출력

7. **Individual result processing(개별 결과 처리)**:
   ```python
   if results:
       best_pred = results['predictions'][0]
       print(f"🏆 {filename}: {best_pred['class_name']} ({best_pred['percentage']:.1f}%)")
       results_list.append((filename, best_pred['class_name'], best_pred['percentage']))
   ```
   - `best_pred` (최고 예측): 가장 높은 confidence(신뢰도)를 가진 prediction(예측)
   - immediate feedback(즉시 피드백): 각 image(이미지)의 결과 즉시 출력
   - `results_list.append()`: tuple format(튜플 형식)으로 결과 저장

8. **Summary report generation(요약 보고서 생성)**:
   ```python
   print(f"\n{'='*60}")
   print("📋 일괄 분류 결과 요약")
   print(f"{'='*60}")
   for filename, pred_class, confidence in results_list:
       print(f"📷 {filename:25} → {pred_class:20} ({confidence:.1f}%)")
   ```
   - comprehensive summary(종합 요약) 출력
   - formatted table(서식화된 표): filename(파일명), prediction(예측), confidence(신뢰도)
   - string formatting(문자열 서식): 정렬된 column layout(열 레이아웃)

9. **Successful completion(성공적 완료)**:
   ```python
   return results_list
   ```
   - all results(모든 결과) 반환
   - tuple list format(튜플 리스트 형식): `(filename, class_name, confidence)`

10. **Exception handling(예외 처리)**:
    ```python
    except Exception as e:
        print(f"❌ 오류 발생: {e}")
        return None
    ```
    - generic error handling(일반 오류 처리)
    - error message(오류 메시지) 출력 후 `None` 반환

**Function workflow(함수 워크플로우):**
1. User interface(사용자 인터페이스) 초기화
2. Multiple file upload(다중 파일 업로드) 실행
3. Upload validation(업로드 검증)
4. Classifier initialization(분류기 초기화)
5. File iteration(파일 반복) 처리:
   - Individual prediction(개별 예측)
   - Immediate result display(즉시 결과 표시)
   - Result storage(결과 저장)
6. Summary report(요약 보고서) 생성
7. Final results(최종 결과) 반환

**Key features(주요 특징):**
- **Batch processing(일괄 처리)**: 여러 images(이미지) 동시 처리
- **Real-time feedback(실시간 피드백)**: 각 image(이미지) 처리 중 progress(진행 상황) 표시
- **Comprehensive summary(종합 요약)**: 모든 결과를 정리된 format(형식)으로 표시
- **Efficient workflow(효율적 워크플로우)**: single classifier instance(단일 분류기 인스턴스) 재사용

**Data structure(데이터 구조):**
- `results_list`: list of tuples(튜플 리스트)
- 각 tuple(튜플): `(filename, predicted_class, confidence_percentage)`

**Use cases(사용 사례):**
- Multiple image classification(다중 이미지 분류)
- Dataset evaluation(데이터셋 평가)
- Bulk image analysis(대량 이미지 분석)

**Return value(반환값):**
- Success case(성공 시): list of tuples(튜플 리스트) with classification results(분류 결과)
- Failure case(실패 시): `None`

In [ ]:
# =============================================================================
# 메인 실행 부분
# =============================================================================

In [ ]:
print("🎯 ImageNet 1000개 클래스 분류기 실행 옵션:")
print("1. 단일 이미지 분류: classify_uploaded_image()")
print("2. 다중 모델 비교: compare_multiple_models()")
print("3. 여러 이미지 일괄 분류: batch_classify()")
print("\n💡 추천: classify_uploaded_image() 로 시작하세요!")
print("\n🧠 사용 가능한 모델: ResNet50, VGG16, EfficientNet, MobileNet 등")
print("📊 분류 가능: 동물, 차량, 음식, 도구, 자연물 등 1000개 카테고리")
print("\n⚡ 준비 완료! 업로드 전용 ImageNet 분류기입니다!")

# 사용자가 직접 함수 호출
classify_uploaded_image()